In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer

In [ ]:
file_path = "/content/drive/MyDrive/lg aimers"

In [ ]:
train_df = pd.read_csv(file_path + "/data_pre.csv")
test_df = pd.read_csv(file_path + "/data_pre_test.csv")

In [ ]:
X = train_df.drop(['임신 성공 여부', 'ID'], axis=1)
y = train_df['임신 성공 여부']
test_df = test_df.drop(['ID'], axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import pandas as pd
import numpy as np

# 사용할 변수 목록
include_columns = [ '시술 시기 코드', '시술 당시 나이', '시술 유형', '배란 자극 여부', '부부 주 불임 원인',
                   '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인',
                    '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증',
                    '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '총 시술 횟수', 'DI 시술 횟수',
                   '총 임신 횟수', '총 생성 배아 수', '미세주입된 난자 수', '이식된 배아 수',
                   '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수',
                   '해동 난자 수', '저장된 신선 난자 수', '기증자 정자와 혼합된 난자 수', '정자 출처',
                   '난자 기증자 나이', '정자 기증자 나이', '난자 혼합 경과일', '배아 이식 경과일']

ordinal_columns = ["시술 당시 나이", "총 시술 횟수", "총 임신 횟수", 'DI 시술 횟수',
                   "난자 기증자 나이", "정자 기증자 나이"]

categorical_columns = ['시술 시기 코드', "시술 유형", "배란 자극 여부", "부부 주 불임 원인", "부부 부 불임 원인",
    '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애',
    '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도',
    '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '정자 출처']

X_filtered = X[include_columns]
test_df_filtered = test_df[include_columns]

# Label Encoding 적용
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X_filtered[col] = le.fit_transform(X_filtered[col].astype(str))
    test_df_filtered[col] = le.transform(test_df_filtered[col].astype(str))
    label_encoders[col] = le  # 나중에 사용 가능하도록 저장

# 순서형 변수 인코딩
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_ordinal = ordinal_encoder.fit_transform(X_filtered[ordinal_columns])
test_ordinal = ordinal_encoder.transform(test_df_filtered[ordinal_columns])

# 데이터프레임 변환
X_ordinal_df = pd.DataFrame(X_ordinal, columns=ordinal_columns, index=X_filtered.index)
test_ordinal_df = pd.DataFrame(test_ordinal, columns=ordinal_columns, index=test_df_filtered.index)

# 기존 수치형 변수들과 결합
X_encoded = pd.concat([X_ordinal_df, X_filtered.drop(columns=ordinal_columns)], axis=1)
test_encoded = pd.concat([test_ordinal_df, test_df_filtered.drop(columns=ordinal_columns)], axis=1)

print("Encoded Train Data:")
print(X_encoded.head())

print("\nEncoded Test Data:")
print(test_encoded.head())


<ipython-input-6-74c50ddddff3>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_filtered[col] = le.fit_transform(X_filtered[col].astype(str))
<ipython-input-6-74c50ddddff3>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_filtered[col] = le.transform(test_df_filtered[col].astype(str))
<ipython-input-6-74c50ddddff3>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

Encoded Train Data:
   시술 당시 나이  총 시술 횟수  총 임신 횟수  DI 시술 횟수  난자 기증자 나이  정자 기증자 나이  시술 시기 코드  \
0       0.0      0.0      0.0       0.0        4.0        6.0         6   
1       5.0      0.0      0.0       0.0        4.0        6.0         5   
2       0.0      1.0      0.0       0.0        4.0        6.0         3   
3       1.0      1.0      0.0       0.0        4.0        6.0         2   
4       0.0      0.0      0.0       0.0        4.0        6.0         3   

   시술 유형  배란 자극 여부  부부 주 불임 원인  ...  이식된 배아 수  저장된 배아 수  미세주입 후 저장된 배아 수  \
0      1         1           0  ...  1.098612  1.098612         1.098612   
1      1         0           0  ...  0.000000  0.000000         0.000000   
2      1         1           0  ...  1.098612  0.000000         0.000000   
3      1         1           0  ...  0.000000  0.000000         0.000000   
4      1         1           0  ...  1.098612  0.000000         0.000000   

   해동된 배아 수  해동 난자 수  저장된 신선 난자 수  기증자 정자와 혼합된 난자 수  정자 출처  난자 혼합 경과일  \

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.6 MB/s eta 0:00:00


In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import joblib

#  랜덤 오버샘플링 정의
ros = RandomOverSampler(random_state=42)

#  HGBT 모델 정의
hgbt = HistGradientBoostingClassifier(random_state=42)

#  CatBoost 모델 정의
catboost = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    learning_rate=0.021,
    depth=6,
    iterations=786,
    l2_leaf_reg=6,
    border_count=122,
    bagging_temperature=0.38,
    random_seed=42,
    verbose=100
)

def feature_engineering(df):
    df = df.copy()
    df["임신 성공률"] = df["총 임신 횟수"] / (df["총 시술 횟수"] + 1e-5)
    df["배아 이식 성공 가능성"] = df["이식된 배아 수"] / (df["총 생성 배아 수"] + 1e-5)
    df["배아 보존율"] = df["저장된 배아 수"] / (df["총 생성 배아 수"] + 1e-5)
    df["정자 혼합 배아 생성률"] = df["기증자 정자와 혼합된 난자 수"] / (df["총 생성 배아 수"] + 1e-5)
    df["배아 해동 성공률"] = df["해동된 배아 수"] / (df["해동 난자 수"] + 1e-5)
    df["시술 후 이식 기간"] = df["배아 이식 경과일"] - df["난자 혼합 경과일"]
    df = df.fillna(0)
    return df

#  Feature Engineering 적용
X_encoded = feature_engineering(X_encoded)
test_encoded = feature_engineering(test_encoded)

#  5-Fold 교차 검증 설정
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
meta_features = np.zeros((len(X_encoded), 2))
cv_auc_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(X_encoded, y)):
    X_train, X_val = X_encoded.iloc[train_index], X_encoded.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

    #  CatBoost 학습
    catboost.fit(X_train_res, y_train_res, verbose=0)
    catboost_pred = catboost.predict_proba(X_val)[:, 1]
    meta_features[val_index, 0] = catboost_pred

    #  HGBT 학습
    hgbt.fit(X_train_res, y_train_res)
    hgbt_pred = hgbt.predict_proba(X_val)[:, 1]
    meta_features[val_index, 1] = hgbt_pred

    #  Soft Voting 앙상블
    ensemble_pred = (hgbt_pred * 0.55) + (catboost_pred * 0.45)
    auc_score = roc_auc_score(y_val, ensemble_pred)
    cv_auc_scores.append(auc_score)

#  메타 모델 입력 데이터 생성
X_meta_train = np.hstack((X_encoded, meta_features))

#  메타 모델 정의 (XGBoost)
xgb_meta = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=42)
xgb_meta.fit(X_meta_train, y)

#  최종 평가
final_auc_score = roc_auc_score(y, xgb_meta.predict_proba(X_meta_train)[:, 1])
print(f'Final Meta Model ROC AUC Score: {final_auc_score}')

#  전체 데이터로 최종 모델 학습
X_res, y_res = ros.fit_resample(X_encoded, y)
hgbt.fit(X_res, y_res)
catboost.fit(X_res, y_res, verbose=0)

#  모델 저장
joblib.dump(hgbt, 'hgbt_model.pkl')
joblib.dump(catboost, 'catboost_model.pkl')
joblib.dump(xgb_meta, 'xgb_meta_model.pkl')
joblib.dump(ros, 'ros_sampler.pkl')
print("HGBT, CatBoost & XGB Meta Model 저장 완료!")

#  테스트 데이터 예측 및 제출 파일 생성
feature_engineering(test_encoded)
test_df_processed = test_encoded.copy()

#  개별 모델 예측 (테스트 데이터)
catboost_pred_test = catboost.predict_proba(test_df_processed)[:, 1]
hgbt_pred_test = hgbt.predict_proba(test_df_processed)[:, 1]
meta_features_test = np.column_stack((catboost_pred_test, hgbt_pred_test))

#  메타 모델 입력 데이터 생성 (테스트 데이터)
X_meta_test = np.hstack((test_df_processed, meta_features_test))

#  메타 모델 예측
meta_pred_test = xgb_meta.predict_proba(X_meta_test)[:, 1]

Final Meta Model ROC AUC Score: 0.7418856081616825
HGBT, CatBoost & XGB Meta Model 저장 완료!


In [ ]:
#  제출 파일 생성
sample_submission = pd.read_csv(file_path + '/sample_submission.csv')
sample_submission['probability'] = meta_pred_test
sample_submission.to_csv(file_path + '/Meta_Model_Submission2.csv', index=False)
print("✅ 최종 제출 파일 './Meta_Model_Submission2.csv' 생성 완료!")


✅ 최종 제출 파일 './Meta_Model_Submission2.csv' 생성 완료!
